In [67]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime

In [2]:
nr_url = 'https://naturalrefrigerants.com/news/'
## seems to work with basic request using headers
nr_article_url = 'https://naturalrefrigerants.com/news/the-middle-east-is-ready-to-scale-natural-refrigerants-says-epta-middle-east-general-manager/'
## seems to work with basic request using headers

all_urls = [nr_url, nr_article_url]

In [19]:
def fetch_method(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        )
    }
    response = requests.get(url, headers=headers, verify=False, timeout=30)
    print(response.status_code)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup

In [23]:
temp_soup = fetch_method(nr_url)
for p in temp_soup.find_all("p"):
    print(p.get_text())

c:\Users\203156\Desktop\Pipelines\Webscraping\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'naturalrefrigerants.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


200
Stay informed with the latest news on natural refrigerants, policies, technology developments, and market trends. We publish original articles and global coverage to keep you ahead in the sustainable cooling space.
By phone:
Europe: (+32) 2 808 90 57United States: (+1) 332 223 4004APAC: (+81) 80 2165 9629
By email:
info@naturalrefrigerants.com
 Username or email address *  
 Password *  
   Remember me  
   Log in 



  
 Lost your password? 
 Email address *  
A link to set a new password will be sent to your email address.
Your personal data will be used to support your experience throughout this website, to manage access to your account, and for other purposes described in our privacy policy.
  Register 
No products in the cart.


In [ ]:

# Find the div where class contains 'ecs-posts' & extract all links
posts_div = temp_soup.find("div", class_=re.compile(r"ecs-posts"))

if posts_div:
    links = [
        a["href"]
        for a in posts_div.find_all("a", class_=re.compile(r"elementor-button"))
        if a.get("href")
    ]
    print(links)
else:
    print("No matching div found")


['https://naturalrefrigerants.com/news/new-york-grassroots-group-leads-successful-refrigerant-recycling-drives/', 'https://naturalrefrigerants.com/news/new-york-grassroots-group-leads-successful-refrigerant-recycling-drives/', 'https://naturalrefrigerants.com/news/german-produce-wholesaler-replaces-r404a-refrigeration-with-61-8kw-frigotec-r290-liquid-chiller/', 'https://naturalrefrigerants.com/news/german-produce-wholesaler-replaces-r404a-refrigeration-with-61-8kw-frigotec-r290-liquid-chiller/', 'https://naturalrefrigerants.com/news/atmo-latam-2026-is-set-for-march-9-10-in-bogota/', 'https://naturalrefrigerants.com/news/atmo-latam-2026-is-set-for-march-9-10-in-bogota/', 'https://naturalrefrigerants.com/news/sanhua-to-offer-complete-packages-of-components-for-co2-and-propane-commercial-refrigeration-systems/', 'https://naturalrefrigerants.com/news/sanhua-to-offer-complete-packages-of-components-for-co2-and-propane-commercial-refrigeration-systems/', 'https://naturalrefrigerants.com/news

Article scraping

In [76]:
def article_scraping_method(link):
    article_html = fetch_method(link)

    new_data = pd.DataFrame(columns=['title', 'summary', 'dateline', 'newslinetext', 'attachmenturl'])

    title = article_html.find("h2", class_=re.compile(r"elementor-heading-title")).get_text()

    container = article_html.find('div', class_='elementor-element elementor-element-0f340ec newslinks elementor-widget elementor-widget-theme-post-content')

    if container:
        temp_wid_cont = container.find_all('div', class_='elementor-widget-container')
        temp_summary = temp_wid_cont[0].get_text().strip()
    else:
        temp_summary = 'no content found'



    info_section = article_html.find('div', class_='elementor-container elementor-column-gap-default')
    publication_date = info_section.find('time').get_text().strip()
    dateline = datetime.strptime(publication_date, '%B %d, %Y')

    content_html = article_html.find('div', class_='elementor-element elementor-element-0f340ec newslinks elementor-widget elementor-widget-theme-post-content')
    article_body = content_html.get_text().strip()

    new_row = {'title':title, 'summary':temp_summary, 'dateline':dateline, 'newslinetext':article_body, 'attachmenturl':link}
    new_data = pd.concat([new_data, pd.DataFrame([new_row])], ignore_index=True)
    return new_data
    
    

In [ ]:
## dataframe defining
## columns: title, summary, publication_date->dateline, article_body->newslinetext, attachmenturl
articles_data = pd.DataFrame(columns=['title', 'summary', 'dateline', 'newslinetext', 'attachmenturl'])


deduped_list = list(set(links))

for link in deduped_list:
    articles_data = pd.concat([articles_data, article_scraping_method(link)])
    

In [83]:
%pip install openpyxl


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [84]:
articles_data.to_excel('scraped_natural_refrigerants_news.xlsx', index=False)